In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [3]:
path = 'CIFAR10/'
cifar10 = torchvision.datasets.CIFAR10(
    path, train=True, download=True, transform=torchvision.transforms.ToTensor())
cifar10_val = torchvision.datasets.CIFAR10(
    path, train=False, download=True, transform=torchvision.transforms.ToTensor())


Files already downloaded and verified
Files already downloaded and verified


In [4]:
# imgs = torch.stack([img_t for img_t, _ in cifar10], dim=3)
# print(imgs.view(3, -1).mean(dim=1))
# print(imgs.view(3, -1).std(dim=1))

In [5]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2470, 0.2435, 0.2616])])


In [6]:
cifar10 = torchvision.datasets.CIFAR10(path, train=True, download=True, transform=transform)
cifar10_val = torchvision.datasets.CIFAR10(path, train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
train_loader = torch.utils.data.DataLoader(
    cifar10, batch_size=4, shuffle=True)
val_loader = torch.utils.data.DataLoader(
    cifar10_val, batch_size=4, shuffle=False)


In [8]:
class Net(nn.Module):
    def __init__(self, n_chansl=32):
        super().__init__()
        self.conv1 = nn.Conv2d(3, n_chansl, 3, padding=1)
        self.conv2 = nn.Conv2d(n_chansl, 8, 3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 10)
        self.pool = nn.MaxPool2d(2, 2)
        
        
    def forward(self, x):
        out = self.pool(F.relu(self.conv1(x)))
        out = F.max_pool2d(F.relu(self.conv2(out)), 2)     
        out = torch.flatten(out,1)
        out = F.tanh(self.fc1(out))
        out = self.fc2(out)
        return out
       

In [9]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs+1):
        loss_train = 0.0

        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            output = model(imgs)
            loss = loss_fn(output, labels)
            print(1)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()
            if epoch == 1 or epoch % 100 == 0:
                print('{} Epoch {}, Training loss {}'.format(epoch,
                                                             loss_train / len(train_loader)))


In [10]:
model = Net().to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(100, optimizer, model, loss_fn, train_loader)


: 

: 